In [61]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from transformers import BitsAndBytesConfig
from datasets import load_from_disk
import torch
import os
import traceback
from pathlib import Path


In [62]:
def load_model_and_tokenizer(model_name: str, local_dir: str = "./model/TinyLlama"):
    if os.path.exists(local_dir) and os.path.isdir(local_dir):
        print(f"🔁 Загрузка модели из локального каталога: {local_dir}")
        tokenizer = AutoTokenizer.from_pretrained(local_dir)
        model = AutoModelForCausalLM.from_pretrained(
            local_dir,
            device_map="auto",
            torch_dtype=torch.float16,
        )
    else:
        print(f"⬇️ Модель не найдена локально, загружаем из интернета и сохраняем в: {local_dir}")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float16,
        )
        # Сохраняем модель и токенизатор в локальный каталог
        tokenizer.save_pretrained(local_dir)
        model.save_pretrained(local_dir)

    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    model.config.use_cache = False
    model.config.pretraining_tp = 1

    return tokenizer, model

In [63]:
def apply_lora(model):
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    model = get_peft_model(model, lora_config)
    model.gradient_checkpointing_enable()
    model.enable_input_require_grads()  # <--- обязательно!

    model.print_trainable_parameters()  # <--- полезно для отладки

    return model


In [64]:
def tokenize_russian_dialogues(batch, tokenizer, max_length=512):
    texts = []

    for question, answer in zip(batch["question"], batch["answer"]):
        if question is None or answer is None:
            continue  # пропускаем пустые

        prompt = question.strip()
        response = answer.strip()

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )

In [65]:
def tokenize_gsm8k_and_MAWPS(batch, tokenizer, max_length=512):
    texts = []

    for question, answer in zip(batch["question"], batch["answer"]):
        if question is None or answer is None:
            continue

        prompt = question.strip()
        response = answer.strip()

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )

In [66]:
def tokenize_asdiv(batch, tokenizer, max_length=512):
    texts = []

    for question, answer in zip(batch["text"], batch["target"]):
        if question is None or answer is None:
            continue

        prompt = question.strip()
        response = answer.strip()

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )


In [67]:
def tokenize_mathqa(batch, tokenizer, max_length=512):
    texts = []

    for question, chain, result in zip(batch["question"], batch["chain"], batch["result"]):
        if question is None or chain is None or result is None:
            continue

        prompt = question.strip()
        response = f"{chain.strip()} → {result.strip()}"

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )


In [68]:
def tokenize_openbookqa(batch, tokenizer, max_length=512):
    texts = []

    for stem, choices, answer_key in zip(batch["question_stem"], batch["choices"], batch["answerKey"]):
        if stem is None or choices is None or answer_key is None:
            continue

        prompt = stem.strip()

        # выбор правильного варианта по answerKey
        label_list = choices["label"]
        text_list = choices["text"]
        label_to_text = {label: text for label, text in zip(label_list, text_list)}
        response = label_to_text.get(answer_key, "").strip()

        if not response:
            continue

        text = (
            "<|system|>\nТы полезный AI-ассистент.\n"
            f"<|user|>\n{prompt}\n"
            f"<|assistant|>\n{response}"
        )
        texts.append(text)

    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )


In [69]:
def tokenize_dataset(dataset, tokenizer, tokenize_fn, max_length=512):
    def wrapped(example):
        return tokenize_fn(example, tokenizer, max_length=max_length)

    return dataset.map(wrapped, batched=True, remove_columns=dataset.column_names)

In [70]:
def train_lora(
    model_name: str,
    dataset_path: str,
    output_dir: str,
    epochs: int = 3,
    batch_size: int = 4,
    lr: float = 1e-5,
    max_length: int = 512,
    tokenize_fn=None
):
    # Загрузка
    tokenizer, model = load_model_and_tokenizer(model_name)
    model = apply_lora(model)
    dataset = load_from_disk(dataset_path)
    dataset = tokenize_dataset(
        dataset,
        tokenizer=tokenizer,
        tokenize_fn=tokenize_fn,
        max_length=max_length
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # обязательно для CausalLM
    )


    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.deterministic = False


    # Аргументы обучения
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=4,
        lr_scheduler_type="cosine",
        warmup_steps=100,
        num_train_epochs=epochs,
        learning_rate=lr,
        fp16=True,
        logging_steps=20,
        save_steps=200,
        save_total_limit=2,
        report_to="none"
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=tokenizer,
        data_collator=data_collator  
    )

    # Обучение
    trainer.train()
    model.save_pretrained(output_dir)

    del model
    del tokenizer
    torch.cuda.empty_cache()

In [71]:
def merge_lora_adapter(
    base_model_path: str,
    adapter_path: str,
    save_path: str
):
    try:
        print("🔄 Загрузка базовой модели и токенизатора")
        tokenizer = AutoTokenizer.from_pretrained(base_model_path)
        model = AutoModelForCausalLM.from_pretrained(
            base_model_path,
            torch_dtype=torch.float16,
            device_map="auto"
        )

        print("🧩 Объединение с LoRA-адаптером")
        model = PeftModel.from_pretrained(
            model,
            adapter_path,
            torch_dtype=torch.float16
        )
        model = model.merge_and_unload()

        print("💾 Сохранение объединённой модели и токенизатора")
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)

        print(f"✅ Объединённая модель сохранена в: {save_path}")

    except Exception as e:
        print(f"❌ Ошибка: {e}")
        traceback.print_exc()

    try:
        print("🧹 Очистка памяти")
        del model
        del tokenizer
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"❌ Ошибка очистки памяти: {e}")
        traceback.print_exc()

In [72]:
train_lora(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    dataset_path="./data/russian_dialogues",
    output_dir="./checkpoints/step_dialogue",
    epochs=3,
    batch_size=4,
    tokenize_fn=tokenize_russian_dialogues
)
merge_lora_adapter(
    base_model_path="TinyLlama/TinyLlama-1.1B-Chat-v1.0",                 # ← локально сохранённая/обновлённая модель
    adapter_path=Path("checkpoints/step_dialogue").resolve(),           # ← путь к обученному адаптеру
    save_path="./model/TinyLlama"                                         # ← перезаписываем модель
)

🔁 Загрузка модели из локального каталога: ./model/TinyLlama


C:\Users\Настя\AppData\Local\Temp\ipykernel_58384\138314128.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


Step,Training Loss
20,1.647300
40,1.644200
60,1.584400
80,1.660600
100,1.584700
120,1.624200
140,1.664500
160,1.641400
180,1.629900
200,1.655000


🔄 Загрузка базовой модели и токенизатора
🧩 Объединение с LoRA-адаптером
💾 Сохранение объединённой модели и токенизатора
✅ Объединённая модель сохранена в: ./model/TinyLlama
🧹 Очистка памяти


In [73]:
train_lora(
    model_name="./model/TinyLlama",                            # ← обновлённая модель после 1-го этапа
    dataset_path="./data/gsm8k_sel",                           # ← путь к сохранённому датасету
    output_dir="./checkpoints/step_math",                      # ← новый чекпоинт для math
    epochs=3,
    batch_size=4,
    tokenize_fn=tokenize_gsm8k_and_MAWPS                       # ← твоя функция токенизации
)
merge_lora_adapter(
    base_model_path="./model/TinyLlama",                       # ← обновлённая после step_dialogue
    adapter_path=Path("./checkpoints/step_math").resolve(),    # ← LoRA второго этапа
    save_path="./model/TinyLlama"                              # ← перезаписываем итоговую модель
)

🔁 Загрузка модели из локального каталога: ./model/TinyLlama
trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


Map: 100%|██████████| 7473/7473 [00:02<00:00, 3647.42 examples/s]
C:\Users\Настя\AppData\Local\Temp\ipykernel_58384\138314128.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,1.813200
40,1.838200
60,1.790200
80,1.795500
100,1.698300
120,1.616200
140,1.510000
160,1.415400
180,1.331100
200,1.254400


🔄 Загрузка базовой модели и токенизатора
🧩 Объединение с LoRA-адаптером
💾 Сохранение объединённой модели и токенизатора
✅ Объединённая модель сохранена в: ./model/TinyLlama
🧹 Очистка памяти


In [74]:
train_lora(
    model_name="./model/TinyLlama",                            
    dataset_path="./data/mawps_sel",                          
    output_dir="./checkpoints/step_mawps",                     
    epochs=3,
    batch_size=4,
    tokenize_fn=tokenize_gsm8k_and_MAWPS                     
)
merge_lora_adapter(
    base_model_path="./model/TinyLlama",                      
    adapter_path=Path("./checkpoints/step_mawps").resolve(),             
    save_path="./model/TinyLlama"                              
)

🔁 Загрузка модели из локального каталога: ./model/TinyLlama
trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


Map: 100%|██████████| 1417/1417 [00:00<00:00, 1910.67 examples/s]
C:\Users\Настя\AppData\Local\Temp\ipykernel_58384\138314128.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,1.523500
40,1.502500
60,1.436800
80,1.374200
100,1.265100
120,1.131400
140,1.057100
160,1.034900
180,1.020900
200,1.011500


🔄 Загрузка базовой модели и токенизатора
🧩 Объединение с LoRA-адаптером
💾 Сохранение объединённой модели и токенизатора
✅ Объединённая модель сохранена в: ./model/TinyLlama
🧹 Очистка памяти


In [75]:
train_lora(
    model_name="./model/TinyLlama",                            
    dataset_path="./data/asdiv_sel",                          
    output_dir="./checkpoints/step_asdiv",                     
    epochs=3,
    batch_size=4,
    tokenize_fn=tokenize_asdiv                     
)
merge_lora_adapter(
    base_model_path="./model/TinyLlama",                      
    adapter_path=Path("./checkpoints/step_asdiv").resolve(),       
    save_path="./model/TinyLlama"                              
)

🔁 Загрузка модели из локального каталога: ./model/TinyLlama
trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


Map: 100%|██████████| 1200/1200 [00:00<00:00, 3562.62 examples/s]
C:\Users\Настя\AppData\Local\Temp\ipykernel_58384\138314128.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,1.962000
40,1.863100
60,1.664100
80,1.462800
100,1.329100
120,1.117600
140,1.111900
160,1.054900
180,1.065300
200,1.100300


🔄 Загрузка базовой модели и токенизатора
🧩 Объединение с LoRA-адаптером
💾 Сохранение объединённой модели и токенизатора
✅ Объединённая модель сохранена в: ./model/TinyLlama
🧹 Очистка памяти


In [76]:
train_lora(
    model_name="./model/TinyLlama",                            
    dataset_path="./data/mathqa_sel",                          
    output_dir="./checkpoints/step_mathqa",                     
    epochs=3,
    batch_size=4,
    tokenize_fn=tokenize_mathqa                     
)
merge_lora_adapter(
    base_model_path="./model/TinyLlama",                      
    adapter_path=Path("./checkpoints/step_mathqa").resolve(),             
    save_path="./model/TinyLlama"                              
)

🔁 Загрузка модели из локального каталога: ./model/TinyLlama
trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


Map: 100%|██████████| 11500/11500 [00:03<00:00, 3573.97 examples/s]
C:\Users\Настя\AppData\Local\Temp\ipykernel_58384\138314128.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,1.353000
40,1.342800
60,1.283700
80,1.193100
100,1.129600
120,1.021300
140,0.927000
160,0.846300
180,0.799900
200,0.765200


🔄 Загрузка базовой модели и токенизатора
🧩 Объединение с LoRA-адаптером
💾 Сохранение объединённой модели и токенизатора
✅ Объединённая модель сохранена в: ./model/TinyLlama
🧹 Очистка памяти


In [77]:
train_lora(
    model_name="./model/TinyLlama",                            
    dataset_path="./data/openbookqa_sel",                          
    output_dir="./checkpoints/step_openbookqa",                     
    epochs=3,
    batch_size=4,
    tokenize_fn=tokenize_openbookqa                     
)
merge_lora_adapter(
    base_model_path="./model/TinyLlama",                      
    adapter_path=Path("./checkpoints/step_openbookqa").resolve(),       
    save_path="./model/TinyLlama"                              
)

🔁 Загрузка модели из локального каталога: ./model/TinyLlama
trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


Map: 100%|██████████| 4957/4957 [00:01<00:00, 3430.05 examples/s]
C:\Users\Настя\AppData\Local\Temp\ipykernel_58384\138314128.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,1.854200
40,1.791000
60,1.690400
80,1.573400
100,1.467400
120,1.395500
140,1.336700
160,1.307900
180,1.306400
200,1.275200


🔄 Загрузка базовой модели и токенизатора
🧩 Объединение с LoRA-адаптером
💾 Сохранение объединённой модели и токенизатора
✅ Объединённая модель сохранена в: ./model/TinyLlama
🧹 Очистка памяти
